初始化

In [2]:
%load_ext Cython

In [6]:
# coding = 'utf-8'
import numpy as np
import pandas as pd

**最初代码，对每个标记计算去除该样本后，按x类型的y值均值，时间复杂度为$O(n^2)$** 23.6 s

In [ ]:
def target_mean_v1(data:pd.DataFrame, y_name:str, x_name:str) -> np.ndarray:
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result

# **Python优化**

**对GroupBy改进，时间复杂度为$O(n)$** 276 ms

In [ ]:
def target_mean_v2(data:pd.DataFrame, y_name:str, x_name:str) -> np.ndarray:
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

**进一步优化重复参数成变量** 158 ms

In [ ]:
def target_mean_v3(data:pd.DataFrame, y_name:str, x_name:str) -> np.ndarray:
    data_shape = data.shape[0]

    result = np.zeros(data_shape)
    value_dict = dict()
    count_dict = dict()
    for i in range(data_shape):
        data_loc_x = data.loc[i, x_name]
        data_loc_y = data.loc[i, y_name]
        if data_loc_x not in value_dict:
            value_dict[data_loc_x] = data_loc_y
            count_dict[data_loc_x] = 1
        else:
            value_dict[data_loc_x] += data_loc_y
            count_dict[data_loc_x] += 1
    for i in range(data_shape):
        data_loc_x = data.loc[i, x_name]
        data_loc_y = data.loc[i, y_name]
        result[i] = (value_dict[data_loc_x] - data_loc_y) / (count_dict[data_loc_x] - 1)
    return result

**尝试优化data.loc** 120 ms

In [ ]:
def target_mean_v4(data:pd.DataFrame, y_name:str, x_name:str) -> np.ndarray:
  data_shape = data.shape[0]
  result = np.zeros(data_shape)
  value_dict = dict()
  count_dict = dict()

  x_val_series = data.loc[:, x_name]
  y_val_series = data.loc[:, y_name]
  for i in range(data_shape):
    data_loc_x = x_val_series[i]
    data_loc_y = y_val_series[i]
    if data_loc_x not in value_dict:
      value_dict[data_loc_x] = data_loc_y
      count_dict[data_loc_x] = 1
    else:
      value_dict[data_loc_x] += data_loc_y
      count_dict[data_loc_x] += 1
  for i in range(data_shape):
      data_loc_x = data.loc[i, x_name]
      data_loc_y = data.loc[i, y_name]
      result[i] = (value_dict[data_loc_x] - data_loc_y) / (count_dict[data_loc_x] - 1) 

  return result

**将data.loc中的数据转换成np.ndarray** 8.62 ms

In [ ]:
def target_mean_v5(data:pd.DataFrame, y_name:str, x_name:str) -> np.ndarray:
  data_shape = data.shape[0]
  result = np.zeros(data_shape)
  value_dict = dict()
  count_dict = dict()

  x_val_array = data.loc[:, x_name].values
  y_val_array = data.loc[:, y_name].values
  for i in range(data_shape):
    data_loc_x = x_val_array[i]
    data_loc_y = y_val_array[i]
    if data_loc_x not in value_dict:
      value_dict[data_loc_x] = data_loc_y
      count_dict[data_loc_x] = 1
    else:
      value_dict[data_loc_x] += data_loc_y
      count_dict[data_loc_x] += 1
  for i in range(data_shape):
    count = count_dict[x_val_array[i]] - 1
    result[i] = (value_dict[x_val_array[i]] - y_val_array[i]) / count

  return result

**将data中的数据直接转换成np.ndarray** 8.56 ms

loc转np.ndarray和直接转np.ndarray其实没有太大的差别，但是最重要的是pandas转numpy后效率的大幅度提升。

In [ ]:
def target_mean_v6(data:pd.DataFrame, y_name:str, x_name:str) -> np.ndarray:
  data_shape = data.shape[0]
  result = np.zeros(data_shape)
  value_dict = dict()
  count_dict = dict()

  x_val_array = data[x_name].values
  y_val_array = data[y_name].values
  for i in range(data_shape):
    data_loc_x = x_val_array[i]
    data_loc_y = y_val_array[i]
    if data_loc_x not in value_dict:
      value_dict[data_loc_x] = data_loc_y
      count_dict[data_loc_x] = 1
    else:
      value_dict[data_loc_x] += data_loc_y
      count_dict[data_loc_x] += 1
  for i in range(data_shape):
    count = count_dict[x_val_array[i]] - 1
    result[i] = (value_dict[x_val_array[i]] - y_val_array[i]) / count

  return result

**此时我觉得python代码在我能力范围内我觉得优化到不错了，我使用cython进行进一步的优化加速** 1.06 ms

# **Cython**

我cdef使用的都是cnp的原生类型

In [ ]:
%%cython -a

import numpy as np
cimport numpy as cnp
import pandas as pd

def target_mean_v7(data:pd.DataFrame, y_name:str, x_name:str) -> np.ndarray: 
  cdef:
    int data_shape = data.shape[0]
    cnp.ndarray[cnp.float64_t] result = np.zeros(data_shape, dtype=np.float64)
    dict value_dict = {}
    dict count_dict = {}
    cnp.ndarray[cnp.int_t] x_val_array = data[x_name].values
    cnp.ndarray[cnp.int_t] y_val_array = data[y_name].values

  for i in range(data_shape):
    data_loc_x = x_val_array[i]
    data_loc_y = y_val_array[i]
    if data_loc_x not in value_dict:
      value_dict[data_loc_x] = data_loc_y
      count_dict[data_loc_x] = 1
    else:
      value_dict[data_loc_x] += data_loc_y
      count_dict[data_loc_x] += 1
  for i in range(data_shape):
    count = count_dict[x_val_array[i]] - 1
    result[i] = (value_dict[x_val_array[i]] - y_val_array[i]) / count

  return result

# **Cython并行**

**我尝试改进算法，并且将cnp的转换成数组指针，这里我将普通的python函数转换成cython的cpdef函数，并使用prange来代替range进行遍历，这是因为我们要解开Gil的锁，来使用并行进行进一步加速，这一次改变的比较大，所以提升很显著** 
60.5 µs

In [ ]:
%%cython -a

import numpy as np
cimport numpy as cnp
import pandas as pd
import cython
cimport cython
from cython.parallel import prange     

cpdef target_mean_v8(data, cnp.str y_name, cnp.str x_name): 
  cdef:
    int data_shape = data.shape[0]
    double[:,] result = np.zeros(data_shape, dtype=np.float64)
    double[:,] value_dict = np.zeros(10, dtype=np.float64)
    double[:,] count_dict = np.zeros(10, dtype=np.float64)
    long[:,] x_val_array = data[x_name].values
    long[:,] y_val_array = data[y_name].values
    int i = 0 

  for i in prange(data_shape, nogil=True):
    value_dict[x_val_array[i]] += y_val_array[i]
    count_dict[x_val_array[i]] += 1
  for i in prange(data_shape, nogil=True):
    result[i] = (value_dict[x_val_array[i]] - y_val_array[i]) / (count_dict[x_val_array[i]] - 1)

  return result

**我们用memoryview代替之前的数组指针，并且用关闭越界检查以及关闭包裹来进一步优化代码** 43 µs

In [ ]:
%%cython -a

import numpy as np
cimport numpy as cnp
import pandas as pd
import cython
cimport cython
from cython.parallel import prange     

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v9(data, cnp.str y_name, cnp.str x_name): 
  cdef:
    int data_shape = data.shape[0]
    double[::1] result = np.zeros(data_shape, dtype=np.float64)
    double[::1] value_dict = np.zeros(10, dtype=np.float64)
    long[::1] count_dict = np.zeros(10, dtype=np.int64)
    long[::1] x_val_array = np.asfortranarray(data[x_name].values, dtype=np.int64)
    long[::1] y_val_array = np.asfortranarray(data[y_name].values, dtype=np.int64)
    int i = 0 
    long x

  for i in prange(data_shape, nogil=True):
    x = x_val_array[i]
    value_dict[x] += y_val_array[i]
    count_dict[x] += 1
  for i in prange(data_shape, nogil=True):
    x = x_val_array[i]
    result[i] = (value_dict[x] - y_val_array[i]) / (count_dict[x] - 1)

  return result

我尝试在变量的数据类型上下功夫，使用指针和上一版的memoryview进行对比 38.3 µs

In [56]:
%%cython -a

import numpy as np
cimport numpy as cnp
import pandas as pd
import cython
cimport cython
from cython.parallel import prange     

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v10(data, cnp.str y_name, cnp.str x_name): 
  cdef:
    int data_shape = data.shape[0]
    double[:,] result = np.zeros(data_shape, dtype=np.float64)
    double[:,] value_dict = np.zeros(10, dtype=np.float64)
    long[:,] count_dict = np.zeros(10, dtype=np.int64)
    long[:,] x_val_array = data[x_name].values
    long[:,] y_val_array = data[y_name].values
    int i = 0 
    long x

  for i in prange(data_shape, nogil=True):
    x = x_val_array[i]
    value_dict[x] += y_val_array[i]
    count_dict[x] += 1
  for i in prange(data_shape, nogil=True):
    x = x_val_array[i]
    result[i] = (value_dict[x] - y_val_array[i]) / (count_dict[x] - 1)

  return result

**在没有新的思路的情况下，我测试了多种组合，发现可能memoryview与数组指针的混用效率更高？（不确定）**

34.5 µs

In [70]:
%%cython -a

import numpy as np
cimport numpy as cnp
import pandas as pd
import cython
cimport cython
from cython.parallel import prange     

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v11(data, cnp.str y_name, cnp.str x_name): 
  cdef:
    int data_shape = data.shape[0]
    double[::1] result = np.zeros(data_shape, dtype=np.float64)
    double[::1] value_dict = np.zeros(10, dtype=np.float64)
    long[::1] count_dict = np.zeros(10, dtype=np.int64)
    long[:,] x_val_array = data[x_name].values
    long[:,] y_val_array = data[y_name].values
    long x
    int i = 0 

  for i in prange(data_shape, nogil=True):
    x = x_val_array[i]
    value_dict[x] += y_val_array[i]
    count_dict[x] += 1
  for i in prange(data_shape, nogil=True):
    x = x_val_array[i]
    result[i] = (value_dict[x] - y_val_array[i]) / (count_dict[x] - 1)

  return result

**还有就是在外面pd转换成np带进来，必然快了不少。** 31.9 µs

In [126]:
%%cython -a

import numpy as np
cimport numpy as cnp
import pandas as pd
import cython
cimport cython
from cython.parallel import prange     

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v12(cnp.ndarray[cnp.int_t] x_val_array, cnp.ndarray[cnp.int_t] y_val_array): 
  cdef:
    int data_shape = x_val_array.shape[0]
    double[::1] result = np.zeros(data_shape, dtype=np.float64)
    double[::1] value_dict = np.zeros(10, dtype=np.float64)
    #cnp.ndarray[cnp.float64_t] value_dict = np.zeros(10, dtype=np.float64)
    long[::1] count_dict = np.zeros(10, dtype=np.int64)
    long x
    int i = 0 

  for i in prange(data_shape, nogil=True):
    x = x_val_array[i]
    value_dict[x] += y_val_array[i]
    count_dict[x] += 1
  for i in prange(data_shape, nogil=True):
    x = x_val_array[i]
    result[i] = (value_dict[x] - y_val_array[i]) / (count_dict[x] - 1)

  return result

让我们跑5000个标记

In [60]:
c = 5000
y = np.random.randint(2, size=(c, 1))
x = np.random.randint(10, size=(c, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

运行时间测试

In [ ]:
%%timeit
target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 23.6 s per loop


In [ ]:
%%timeit
target_mean_v2(data, 'y', 'x')

1 loop, best of 3: 276 ms per loop


In [ ]:
%%timeit
target_mean_v3(data, 'y', 'x')

10 loops, best of 3: 158 ms per loop


In [ ]:
%%timeit
target_mean_v4(data, 'y', 'x')

10 loops, best of 3: 120 ms per loop


In [ ]:
%%timeit
target_mean_v5(data, 'y', 'x')

100 loops, best of 3: 8.62 ms per loop


In [ ]:
%%timeit
target_mean_v6(data, 'y', 'x')

100 loops, best of 3: 8.56 ms per loop


In [ ]:
%%timeit
target_mean_v7(data, 'y', 'x')

1000 loops, best of 3: 1.06 ms per loop


In [ ]:
%%timeit
target_mean_v8(data, 'y', 'x')

The slowest run took 12.16 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 60.7 µs per loop


In [ ]:
%%timeit
target_mean_v9(data, 'y', 'x')

The slowest run took 4.95 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 43 µs per loop


In [61]:
%%timeit
target_mean_v10(data, 'y', 'x')

The slowest run took 160.28 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 42.7 µs per loop


In [128]:
%%timeit
target_mean_v11(data, 'y', 'x')

The slowest run took 22.42 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 34.5 µs per loop


In [127]:
%%timeit
target_mean_v12(data["x"].values, data["y"].values)

The slowest run took 28.36 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 31.9 µs per loop


进行误差测试

In [ ]:
import numpy as np

def is_norm(x, y): return np.linalg.norm(x - y)
v_nums = 12

for i in range(2, v_nums+1):
  traget_array = eval(f"target_mean_v{i}(data, 'y', 'x')")
  print(f"target_mean_v{i}:{is_norm(result_1, traget_array)}")


target_mean_v2:0.0
target_mean_v3:0.0
target_mean_v4:0.0
target_mean_v5:0.0
target_mean_v6:0.0
target_mean_v7:0.0
target_mean_v8:0.0
target_mean_v9:0.0
target_mean_v10:0.0
target_mean_v11:0.0
